In [1]:
import shared_variables_robot
import socket
import threading
import CAN_ID
import can
import numpy as np
import time
import json

In [ ]:
main_loop_event = threading.Event()

def start_wifi():
    global client

    PORT = 5050
    ADDR = (shared_variables_robot.ip_dico["Matthieu"],PORT)
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(ADDR)
    client.settimeout(0.5)

def receive():
    global client

    received_data = None
    waypoints = []
    prec_waypoints = None
    
    while main_loop_event.is_set():
        with shared_variables_robot.client_lock:
            try :
                received_data = client.recv(1024).decode('utf-8')
            except socket.timeout:
                received_data = None
                continue
            
            if received_data != None:
                if received_data[0] == "[":
                    waypoints = json.loads(received_data)
                    if waypoints != prec_waypoints:
                        compteur = 0
                        for points in waypoints:
                            
                            send_CAN([points,compteur,len(waypoints)])
                            compteur += 1
                            
                        prec_waypoints = waypoints
                        received_data = None
                else:
                    if received_data == "STOP_RPI":
                        main_loop_event.clear()
                        client.close()
                    if received_data == "STOP":
                        send_CAN(CAN_ID.STOP_ROBOT_FIN_MATCH)
                    if received_data == "START":
                        main_loop_event.set()
            

def send():
    global client

    while main_loop_event.is_set():
        can_msg = receive_CAN()

        if can_msg is not None:
            with shared_variables_robot.client_lock:
                if can_msg == True :
                    msg = "Done"
                    client.send(msg.encode('utf-8'))

                can_msg = None

def send_CAN(msg):
    with shared_variables_robot.CAN_lock:
        if type(msg) == list:

            byte_convert_x = msg[0][0].to_bytes(2, byteorder="big")
            last_x, first_x = byte_convert_x

            byte_convert_y = msg[0][1].to_bytes(2, byteorder="big")
            last_y, first_y = byte_convert_y

            bus.send(can.Message(arbitration_id=CAN_ID.POLAIRE, data=[msg[1],
                                                                       np.uint8(last_x), np.uint8(first_x), np.uint8(last_y),
                                                                       np.uint8(first_y),msg[2],0,0], is_extended_id=False))
            print("Message envoyé")
            time.sleep(0.015)
        elif msg == CAN_ID.STOP_ROBOT_FIN_MATCH:
            bus.send(can.Message(arbitration_id=CAN_ID.STOP_ROBOT_FIN_MATCH, data=[1,0,0,0,0,0,0,0], is_extended_id=False))

def receive_CAN():
    with shared_variables_robot.CAN_lock:
        message = bus.recv(timeout=0.015)
        if message is None:
            return None
        else:
            if message.arbitration_id == CAN_ID.ACKNOWLEDGE_BASE_ROULANTE:
                return True
        return None

if __name__ == "__main__":
    bus = can.interface.Bus(channel='can0', interface='socketcan', bitrate=1000000)
    #bus.send(can.Message(arbitration_id=CAN_ID.STOP_ROBOT_FIN_MATCH, data=[1,0,0,0,0,0,0,0], is_extended_id=False))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

    start_wifi()

    main_loop_event.set()

    t_receive = threading.Thread(target = receive)
    t_send = threading.Thread(target = send)

    t_receive.start()
    t_send.start()  

    t_receive.join()
    t_send.join()

Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
Message envoyé
